
# Decision Tree 
Like **SVMs**, **Decision Trees** are versatile Machine Learning algorithms that can perform both
**classification** and **regression** tasks, and even multioutput tasks. They are very powerful algorithms,
capable of fitting complex datasets.

**Decision Trees** are also the fundamental components of **Random Forests**, which are among the most powerful Machine Learning algorithms available today.

### Training and Visualizing a Decision Tree
To understand Decision Trees, let’s just build one and take a look at how it makes predictions. The
following code trains a DecisionTreeClassifier on the iris dataset

In [23]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

You can visualize the trained Decision Tree by first using the export_graphviz() method to output a
graph definition file called iris_tree.dot:

In [24]:
from sklearn.tree import export_graphviz
export_graphviz(
tree_clf,
out_file="iris_tree.dot",
feature_names=iris.feature_names[2:],
class_names=iris.target_names,
rounded=True,
filled=True )

Then you can convert this .dot file to a variety of formats such as PDF or PNG using the dot commandline
tool from the graphviz package.1 This command line converts the .dot file to a .png image file:

In [22]:
import pydot
(graph,) = pydot.graph_from_dot_file('iris_tree.dot')
#graph.write_png('iris_tree.png')
# It is like I need to find a right way to set a path for graphviz for it to find dot.exe
# C:\Program Files (x86)\Graphviz2.38\bin

[]

In [18]:
#from subprocess import check_call
#check_call(['dot','-Tpng','iris_tree.dot','-o','iris_tree.png'])

One of the many qualities of Decision Trees is that they require very little data preparation. In particular, they don’t require
feature scaling or centering at all.


### Making Predictions
A node’s samples attribute counts how **many training instances** it applies to. For example, 100 training
instances have a petal length greater than 2.45 cm (depth 1, right), among which 54 have a petal width
smaller than 1.75 cm (depth 2, left). A node’s value attribute tells you how many training instances of
each class this node applies to: for example, the bottom-right node applies to 0 Iris-Setosa, 1 Iris-
Versicolor, and 45 Iris-Virginica.
Finally, a node’s **gini** attribute measures its **impurity**: a node is “pure” (gini=0) if all training instances it applies to belong to the same class. For example, since the depth-1 left node applies only to Iris-Setosa training instances, it is pure and its gini score is 0. shows how the training algorithm computes the gini score Gi of the ith node. For example, the depth-2 left node has a **gini score** equal to **1 – (0/54)2 – (49/54)2 – (5/54)2 ≈ 0.168**. 

### Estimating Class Probabilities
A Decision Tree can also estimate the probability that an instance belongs to a particular class k: first it
traverses the tree to find the leaf node for this instance, and then it returns the ratio of training instances of
class k in this node. For example, suppose you have found a flower whose petals are 5 cm long and 1.5
cm wide. The corresponding leaf node is the depth-2 left node, so the Decision Tree should output the
following probabilities: 0% for Iris-Setosa (0/54), 90.7% for Iris-Versicolor (49/54), and 9.3% for Iris-
Virginica (5/54). And of course if you ask it to predict the class, it should output Iris-Versicolor (class 1)
since it has the highest probability. Let’s check this:

In [26]:
tree_clf.predict_proba([[5, 1.5]])
#array([[ 0. , 0.90740741, 0.09259259]])
tree_clf.predict([[5, 1.5]])

array([1])

### The CART Training Algorithm
Scikit-Learn uses the Classification And Regression Tree (CART) algorithm to train Decision Trees (also
called “growing” trees). The idea is really quite simple: the algorithm first splits the training set in two
subsets using a single feature k and a threshold tk (e.g., “petal length ≤ 2.45 cm”). How does it choose k
and tk? It searches for the pair (k, tk) that produces the purest subsets (weighted by their size).


Once it has successfully split the training set in two, it splits the subsets using the same logic, then the subsubsets
and so on, recursively. It stops recursing once it reaches the maximum depth (defined by the max_depth hyperparameter), or if it cannot find a split that will reduce impurity. A few other hyperparameters (described in a moment) control additional stopping conditions (min_samples_split, min_samples_leaf, min_weight_fraction_leaf, and max_leaf_nodes).

### Warning
As you can see, the CART algorithm is a greedy algorithm: it greedily searches for an optimum split at the top level, then
repeats the process at each level. It does not check whether or not the split will lead to the lowest possible impurity several levels
down. A greedy algorithm often produces a reasonably good solution, but it is not guaranteed to be the optimal solution.

### Gini Impurity or Entropy?
By default, the Gini impurity measure is used, but you can select the entropy impurity measure instead by
setting the criterion hyperparameter to "entropy". The concept of entropy originated in
thermodynamics as a measure of molecular disorder: entropy approaches zero when molecules are still
and well ordered. It later spread to a wide variety of domains, including Shannon’s information theory,
where it measures the average information content of a message:4 entropy is zero when all messages are
identical. In Machine Learning, it is frequently used as an impurity measure: a set’s entropy is zero when
it contains instances of only one class.

Hi = - sum (Pik * log(Pik)) 

So should you use Gini impurity or entropy? The truth is, most of the time it does not make a big
difference: they lead to similar trees. Gini impurity is slightly faster to compute, so it is a good default.
However, when they differ, Gini impurity tends to isolate the most frequent class in its own branch of the
tree, while entropy tends to produce slightly more balanced trees.


### Regularization Hyperparameters
Decision Trees make very few assumptions about the training data (as opposed to linear models, which
obviously assume that the data is linear, for example). If left unconstrained, the tree structure will adapt
itself to the training data, fitting it very closely, and most likely overfitting it. Such a model is often called
a nonparametric model, not because it does not have any parameters (it often has a lot) but because the
number of parameters is not determined prior to training, so the model structure is free to stick closely to
the data. In contrast, a parametric model such as a linear model has a predetermined number of
parameters, so its degree of freedom is limited, reducing the risk of overfitting (but increasing the risk of
underfitting).
To avoid overfitting the training data, you need to restrict the Decision Tree’s freedom during training. As
you know by now, this is called regularization. The regularization hyperparameters depend on the
algorithm used, but generally you can at least restrict the maximum depth of the Decision Tree. In Scikit-
Learn, this is controlled by the max_depth hyperparameter (the default value is None, which means
unlimited). Reducing max_depth will regularize the model and thus reduce the risk of overfitting.
The DecisionTreeClassifier class has a few other parameters that similarly restrict the shape of the
Decision Tree: min_samples_split (the minimum number of samples a node must have before it can be
split), min_samples_leaf (the minimum number of samples a leaf node must have),
min_weight_fraction_leaf (same as min_samples_leaf but expressed as a fraction of the total
number of weighted instances), max_leaf_nodes (maximum number of leaf nodes), and max_features
(maximum number of features that are evaluated for splitting at each node). Increasing min_*
hyperparameters or reducing max_* hyperparameters will regularize the model.

#### Note
Other algorithms work by first training the Decision Tree without restrictions, then pruning (deleting) unnecessary nodes. A node
whose children are all leaf nodes is considered unnecessary if the purity improvement it provides is not statistically significant.
Standard statistical tests, such as the χ2 test, are used to estimate the probability that the improvement is purely the result of
chance (which is called the null hypothesis). If this probability, called the p-value, is higher than a given threshold (typically 5%,
controlled by a hyperparameter), then the node is considered unnecessary and its children are deleted. The pruning continues until
all unnecessary nodes have been pruned.

## Regression
Decision Trees are also capable of performing regression tasks. Let’s build a regression tree using Scikit-
Learn’s DecisionTreeRegressor class, training it on a noisy quadratic dataset with max_depth=2:

In [27]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')